# 导入模块

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

In [44]:
pd.set_option('display.max_columns', None)

# 定义数据库连接

In [45]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [46]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method
from  db_digua_business.t_order  om 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -2 day )
-- and  om.create_time <= CURRENT_DATE   -- 近一天数据
;
'''

df_order = query(sql1)
df_order.shape

(12059, 29)

# 合并数据

In [47]:
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(12059)

# 处理日期

In [48]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df.shape

(12059, 31)

In [49]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                                                             6020
冰鉴火眸分<500命中强拒                                                                                                1962
命中强拒加入临时黑名单                                                                                                   710
年龄超过49岁或低于18岁                                                                                                 254
命中非免押用户拒绝                                                                                                     180
                                                                                                             ... 
命中云商分低于515强拒, 履约历史等级=1强拒, 命中自有模型强拒, 命中特殊策略2强拒                                                                   1
命中云商分低于515强拒, 命中策略240703_4强拒                                                                                    1
命中云商分低于515强拒, 命中融安分低于680强拒, 评分等级低于D,命中拒绝, 命中融安分<700, 命中青云分<500, 命中策略240703_14强拒, 命中

# 处理备注信息

In [50]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [51]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    11530
1      529
Name: count, dtype: int64

# 归属渠道

In [52]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"    
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                   6791
搜索渠道                   2149
八派信息                    536
小程序商家消息（服务提醒）           533
付费灯火                    436
付费邦道-首页                 287
我的小程序入口                 238
单人聊天会话中的小程序消息卡片（分享）     220
小舞直播                    151
九州信息                    135
其他渠道场景渠道。               126
支付宝 push 消息（同1014）       88
未知渠道                     76
支付宝社群                    68
抖音渠道                     48
生活号                      39
数立月活SL06                 32
分期乐2                     25
邦道-销售线索                  17
邦道-自定义                   16
支付成功页                    13
数立月活SL10                 10
曙光计划                      5
白名单客户                     5
支付宝客户端首页                  3
扫描二维码                     3
激励广告(抖音极速版)               3
卡包\r\n                    1
线下易拉宝                     1
数立月活SL09                  1
设置页面入口(抖音极速版)             1
分期乐1                      1
首页十二宫格及更多                 1
Name: count, dtype: int64

# 订单去重

In [53]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [54]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [55]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
进件前取消    5121
前置拦截     2598
机审强拒     1122
无法联系      379
客户取消      183
人审拒绝       82
Name: count, dtype: int64

# 保留商家信息

In [56]:
df_contain = df.copy()

# 剔除商家数据

In [57]:
#剔除商家数据只保留自营租机业务数据
df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)
df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)  
# 趣智数码  单
df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)
# df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)


# 去重订单明细导出

In [58]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2024-09-27'

# 导入订单服务

In [59]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(1632, 52)

In [60]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)



In [61]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'status2', 'channel_type', 'sku_attributes', 'product_name',
       'new_actual_money', 'back_type', 'user_mobile', 'true_name',
       'id_card_num', 'total_describes', 'decision_result', 'cancel_reason',
       'trace_id', 'rejected', 'result', 'channel_name', 'activity_name',
       'merchant_id', 'merchant_name', 'total_freeze_fund_amount',
       'buy_service_product', 'service_status', 'order_method', '下单日期', '月份',
       '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案', '押金类型', '优惠券使用否',
       '是否进行预授权', '颜色', '内存', '进件', '来源渠道', '免审', '归属渠道', '状态编码', '是否前置拦截',
       '是否机审强拒', '是否出库前风控强拒', '审核状态', '下单时段', '待审核', '前置拦截', '机审强拒', '人审拒绝',
       '客户取消', '出库前风控强拒', '无法联系', '是否进件', '是否出库', '进件前取消'],
      dtype='object')

# 取消订单

In [62]:
# 进件订单
jj_order = df.query('是否进件==1')
# 机审强拒订单
jsqj_order = df.query('机审强拒==1')
# 获取机jj_order中不在jsqj_order的订单
jsqj_unique = jj_order.index.difference(jsqj_order.index)
# 使用这些字段重新索引jj_order
jsqj_orders = jj_order.reindex(jsqj_unique)

# 出库订单
ck_order = df.query('是否出库==1')
# 获取机df_jsqj_order中不在ck_order的订单
jsqj_unique = jsqj_orders.index.difference(ck_order.index) 
# 使用这些字段重新索引df_jsqj_order
jsqj_orders = jsqj_orders.reindex(jsqj_unique) 

# # 出库前风控强拒订单
df_fkqj_order = df.query('出库前风控强拒==1')
# # 获取机jsqj_order中不在df_fkqj_order的订单
jsqj_unique = jsqj_orders.index.difference(df_fkqj_order.index) 
# # 使用这些字段重新索引df_jsqj_order
jsqj_orders = jsqj_orders.reindex(jsqj_unique) 

# # 人审拒绝订单
rs_order = df.query('人审拒绝==1')
# # 获取机jsqj_order中不在rs_order的订单
jsqj_unique = jsqj_orders.index.difference(rs_order.index) 
# # 使用这些字段重新索引df_jsqj_order
jsqj_orders = jsqj_orders.reindex(jsqj_unique) 
# jsqj_orders

order_qx = jsqj_orders.query('status2=="已退款"')[['create_time', 'order_number', 'cancel_reason']]
order_qx.rename(columns={'create_time': '订单日期', 'order_number': '订单号', 'cancel_reason': '取消原因'}, inplace=True)
Today1 = str(datetime.now().strftime('%Y%m%d%H'))
with pd.ExcelWriter(f'F:/需求/取消订单数据_{Today1}.xlsx', engine='openpyxl') as writer:
    order_qx.to_excel(writer, sheet_name='取消订单', index=False)

In [63]:
df_rsjj = df[df['人审拒绝']==1]
df_rsjj = df_rsjj[['create_time', 'order_number', '电审拒绝原因']]
df_rsjj.rename(columns={'create_time': '订单日期', 'order_number': '订单号'}, inplace=True)
# df_rsjj
# with pd.ExcelWriter(f'F:/需求/取消订单数据_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#     df_rsjj.to_excel(writer, sheet_name='人审拒绝', index=False)

,订单日期,订单号,电审拒绝原因
208,2024-09-25 01:13:32,A2024092501133129,"拒绝电审""]"
937,2024-09-25 08:28:08,A2024092508280750,"短期在租多台""]"
964,2024-09-25 08:40:23,A2024092508402279,"短期在租多台""]"
1131,2024-09-25 09:41:22,A20240925094122119,"第三方指导电审""]"
1198,2024-09-25 09:55:00,A20240925095459190,"短期在租多台""]"
1456,2024-09-25 10:47:30,A20240925104730242,"短期在租多台""]"
1475,2024-09-25 10:50:16,A20240925105015261,"多拍/下单轨迹异常""]"
2039,2024-09-25 12:32:51,A20240925123250173,"拒绝电审""]"
2380,2024-09-25 13:50:18,A20240925135018226,"短期在租多台""]"
2408,2024-09-25 13:54:51,A20240925135451254,"命中高风险地址""]"


In [64]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

H = datetime.now().strftime('%H')
M = datetime.now().strftime('%M')
def my_job1():
    print(f'执行定时任务：现在是每日的{H}点{int(M)+1}分')
    path1 = r'\\digua\迪瓜租机\13.运营-信审每日数据/' # 共享盘 \\digua
    # path1 = 'F:/需求/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    with pd.ExcelWriter(path1 + f'取消订单数据_{Today1}.xlsx', engine='openpyxl') as writer:
        order_qx.to_excel(writer, sheet_name='取消订单', index=False)
    with pd.ExcelWriter(path1 + f'取消订单数据_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
        df_rsjj.to_excel(writer, sheet_name='电审拒绝', index=False)
scheduler1 = BackgroundScheduler()
# 添加一个cron任务，每天的16点55分执行
# scheduler1.add_job(my_job1, 'cron', day_of_week='0', hour=H, minute=int(M)+1)
scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)

# 启动scheduler1
scheduler1.start()
    

执行定时任务：现在是每日的15点39分
执行定时任务：现在是每日的15点39分
